In [3]:
import pandas as pd
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_extraction.text import CountVectorizer
df = pd.read_csv('/content/spam.csv', encoding = 'latin-1')
df = df[['v1','v2']]
df.columns = ['label', 'message']
print(df.head())
print(df.shape)
print(df['label'].value_counts())
print(df['label'].unique())
df['label'] = df['label'].map({'ham':0, 'spam':1})
print(df.head())
print(df['label'].value_counts())

def clean_text(text):
  text = text.lower()
  text = re.sub(r'\d+', '', text)
  text = text.translate(str.maketrans('', '', string.punctuation))
  text = text.strip()
  return text

x = df['message']
y = df['label']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)
print(x_train.shape)
print(x_test.shape)
vectorizer = CountVectorizer(stop_words = 'english')
x_train_bow = vectorizer.fit_transform(x_train)
x_test_bow = vectorizer.transform(x_test)
print(x_train_bow.shape)
print(x_test_bow.shape)
bow_model = MultinomialNB()
bow_model.fit(x_train_bow, y_train)

y_pred_bow = bow_model.predict(x_test_bow)

print("Bag of Words Accuracy:", accuracy_score(y_test, y_pred_bow))
print(confusion_matrix(y_test, y_pred_bow))
print(classification_report(y_test, y_pred_bow))


# -------------------- TF-IDF --------------------
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
X_train_tfidf = tfidf_vectorizer.fit_transform(x_train)
X_test_tfidf = tfidf_vectorizer.transform(x_test)

tfidf_model = MultinomialNB()
tfidf_model.fit(X_train_tfidf, y_train)

y_pred_tfidf = tfidf_model.predict(X_test_tfidf)

print("TF-IDF Accuracy:", accuracy_score(y_test, y_pred_tfidf))
print(confusion_matrix(y_test, y_pred_tfidf))
print(classification_report(y_test, y_pred_tfidf))


# -------------------- Custom Prediction --------------------
sample_message = ["Congratulations! You have won a free gift card. Click now"]
sample_message_cleaned = [clean_text(sample_message[0])]
sample_vector = tfidf_vectorizer.transform(sample_message_cleaned)

prediction = tfidf_model.predict(sample_vector)

if prediction[0] == 1:
    print("Spam Message")
else:
    print("Not Spam Message")

  label                                            message
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...
(5572, 2)
label
ham     4825
spam     747
Name: count, dtype: int64
['ham' 'spam']
   label                                            message
0      0  Go until jurong point, crazy.. Available only ...
1      0                      Ok lar... Joking wif u oni...
2      1  Free entry in 2 a wkly comp to win FA Cup fina...
3      0  U dun say so early hor... U c already then say...
4      0  Nah I don't think he goes to usf, he lives aro...
label
0    4825
1     747
Name: count, dtype: int64
(4457,)
(1115,)
(4457, 7472)
(1115, 7472)
Bag of Words Accuracy: 0.9838565022421525
[[959   6]
 [ 12 138]]
              precision    recall  f1-score 